<a href="https://colab.research.google.com/github/markNZed/GPT-NeoX-Colab/blob/main/notebooks/shakespeare_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Experiment
This is a demonstration of how experiments can be run using DagsHub and MLflow.
We will train three different versions of the tiny LLM using different batch sizes and compare the results.

## ToDo
- Shorten the training time for testing
- Run tests in parallel
- Extract functions

## Login to Dagshub
To avoid requirest in the middle of the experiment

In [15]:
import os
from google.colab import userdata
#os.environ["MLFLOW_TRACKING_PASSWORD"] = userdata.get("MLFLOW_TRACKING_PASSWORD")
#os.environ["MLFLOW_TRACKING_USERNAME"] = userdata.get("MLFLOW_TRACKING_USERNAME")
os.environ["DAGSHUB_USER_TOKEN"] = userdata.get("DAGSHUB_USER_TOKEN")
if not os.environ["DAGSHUB_USER_TOKEN"]:
  dagshub.auth.get_token()

In [2]:
#@title Setup paths
# We could modify these paths to "stub" behavior for test/dev
# Use functions so if the Notebook is restarted we do not lose the values
def workspaceDir():
    return "/content"
def GPTNeoXDirName():
    return "gpt-neox"
def GPTNeoXDir():
    return f"{workspaceDir()}/{GPTNeoXDirName()}"
def GPTNeoXColabDirName():
    return "GPT-NeoX-Colab"
def GPTNeoXColabDir():
    return f"{workspaceDir()}/{GPTNeoXColabDirName()}"

In [3]:
#@title Clone GPT-NeoX-Colab
%%time
%cd {workspaceDir()}
# Don't use --depth 1 because that does not play nice with git-annex
!git clone https://github.com/markNZed/GPT-NeoX-Colab.git
%cd {GPTNeoXColabDir()}
%pip install -q -r requirements_colab.txt
%pip install -q .
from dotenv import load_dotenv
import os
load_dotenv(f"{GPTNeoXColabDir()}/.env")
import GPTNeoXColab
GPTNeoXColab.utils.colab.fetch_data("data/shakespeare/shakespeare_text_document.bin")
GPTNeoXColab.utils.colab.fetch_data("data/shakespeare/shakespeare_text_document.idx")
!mkdir -p {GPTNeoXDir()}/processed_data
!cp {GPTNeoXColabDir()}/data/shakespeare/shakespeare_text_document.* {GPTNeoXDir()}/processed_data

/content
Cloning into 'GPT-NeoX-Colab'...
remote: Enumerating objects: 1129, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 1129 (delta 35), reused 41 (delta 19), pack-reused 1052 (from 1)
Receiving objects: 100% (1129/1129), 13.78 MiB | 10.22 MiB/s, done.
Resolving deltas: 100% (612/612), done.
/content/GPT-NeoX-Colab
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.5/251.5 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 88.2 MB/s eta 

In [4]:
#@title Clone GPT-NeoX
%%time
%cd {workspaceDir()}
!git clone --depth 1 https://github.com/EleutherAI/gpt-neox

/content
Cloning into 'gpt-neox'...
remote: Enumerating objects: 296, done.
remote: Counting objects: 100% (296/296), done.
remote: Compressing objects: 100% (231/231), done.
remote: Total 296 (delta 74), reused 136 (delta 43), pack-reused 0 (from 0)
Receiving objects: 100% (296/296), 2.50 MiB | 6.23 MiB/s, done.
Resolving deltas: 100% (74/74), done.
CPU times: user 16.4 ms, sys: 1.31 ms, total: 17.7 ms
Wall time: 1.01 s


In [5]:
#@title Load prebuilt Python environment for Colab
%%time
%cd {workspaceDir()}
GPTNeoXColab.utils.colab.download_my_env()

/content
Unzipping my_env.tar.gz
Untarring my_env.tar.gz
CPU times: user 10.6 s, sys: 8.75 s, total: 19.4 s
Wall time: 2min 31s


# Run Experiment

In [6]:
import GPTNeoXColab
import os
from pathlib import Path
ROOT_DIR = GPTNeoXColab.utils.colab.find_project_root()
RELATIVE_ROOT_DIR = os.path.relpath(ROOT_DIR, Path.cwd())

In [7]:
experiment_name = "experiment1"  # Change this to dynamically load different experiments

In [21]:
import dagshub
dagshub.auth.add_app_token(token=os.environ["DAGSHUB_USER_TOKEN"])

In [28]:
#@title Run the training in a detached background process
%cd {workspaceDir()}
import subprocess
import os
import dagshub
import mlflow
from omegaconf import DictConfig, OmegaConf
from hydra.core.global_hydra import GlobalHydra
from hydra import initialize, compose

# Clear Hydra's global state if it’s already initialized
if GlobalHydra.instance().is_initialized():
    GlobalHydra.instance().clear()

initialize(config_path=f"{RELATIVE_ROOT_DIR}/configs", version_base="1.1")

cfg = compose(config_name="hydra", overrides=[f"experiments={experiment_name}"])

# Set MLflow tracking URI for DagsHub
#MLFLOW_TRACKING_URI = f"https://dagshub.com/{cfg.dagshub.repo_owner}/{cfg.dagshub.repo_name}.mlflow"

os.environ["DAGSHUB_USER"] = cfg.dagshub.repo_owner
os.environ["DAGSHUB_TOKEN"] = ""

# Initialize DagsHub logging
try:
    # Will setup MLFLOW_TRACKING_URI MLFLOW_TRACKING_USERNAME MLFLOW_TRACKING_PASSWORD
    dagshub.init(repo_owner=cfg.dagshub.repo_owner, repo_name=cfg.dagshub.repo_name, mlflow=True)
except Exception as e:
    print(f"Failed to initialize DagsHub logging: {e}")

def train_model(cfg: DictConfig):
    print("Running experiment:", cfg.experiment_name)
    print(OmegaConf.to_yaml(cfg))

    # Log parameters
    mlflow.log_params(OmegaConf.to_container(cfg, resolve=True))

    %cd {GPTNeoXDir()}
    # The deepy.py script assumes it is running in the root of GTP-NeoX repo
    # Start a detached background process
    process = subprocess.Popen(
        f"nohup bash -c \"source {workspaceDir()}/my_env/bin/activate && python ./deepy.py train.py --conf_dir {GPTNeoXColabDir()}/configs shakespeare shakespeare_deepy\" & echo $! > train_process.pid",
        shell=True,
        executable='/bin/bash',
        preexec_fn=subprocess.os.setsid  # Starts the process in a new session so interrupting Notebook does not kill the training
    )

    print("Training complete.")

exp_id = GPTNeoXColab.utils.ml.get_or_create_experiment_id("tutorial")

# Set the experiment name and start the run
# mlflow.set_experiment(experiment_name)
with mlflow.start_run(experiment_id=exp_id):
    train_model(cfg)


/content


Initialized MLflow to track repo "MarkNZed/GPT-NeoX-Colab"

Repository MarkNZed/GPT-NeoX-Colab initialized!

Running experiment: base_experiment
dagshub:
  repo_owner: MarkNZed
  repo_name: GPT-NeoX-Colab
output_dir: ../outputs
experiment_name: base_experiment
seed: 42
model:
  hidden_dim: 64
training:
  batch_size: 32
  learning_rate: 0.001
  epochs: 2
data: null
experiments:
  experiment_name: experiment_1
  seed: 123

/content/gpt-neox
Training complete.


2024/11/11 11:22:14 INFO mlflow.tracking._tracking_service.client: 🏃 View run thundering-bee-987 at: https://dagshub.com/MarkNZed/GPT-NeoX-Colab.mlflow/#/experiments/3/runs/21f714bd429e48ffbe7fb31a1626c73d.
2024/11/11 11:22:14 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/MarkNZed/GPT-NeoX-Colab.mlflow/#/experiments/3.


In [ ]:
#@title Wait until logs directory is created
import time
import os

# Path to the log directory
logs_dir = f"{GPTNeoXDir()}/logs"

# Wait for the directory to be created
while not os.path.exists(logs_dir):
    print("Waiting for logs directory to be created...")
    time.sleep(10)  # Check every X seconds

print("ogs directory found")

In [33]:
#@title Find the latest log file
import glob
import os

# Define the log directory and pattern for log files
log_dir = f"{GPTNeoXDir()}/logs"
log_pattern = os.path.join(log_dir, "*_stdout.txt")

# Get the list of log files that match the pattern
log_files = glob.glob(log_pattern)

# Ensure there are log files in the directory
if log_files:
    # Find the latest log file based on modification time
    latest_log = max(log_files, key=os.path.getmtime)
    print("Latest log file:", latest_log)
else:
    latest_log = None
    print("No log files found.")


Latest log file: /content/gpt-neox/logs/d1d8af4f6537_stdout.txt


In [ ]:
#@title Read the latest log file and extract the iteration count
import time
import os
import re

# File to store the last read position (persistence between script runs)
file_position = 0
# Regular expression to match "iteration <number> / <total>"
iteration_pattern = re.compile(r"iteration\s+(\d+)\s*/\s*\d+")

def read_new_iterations():
    global file_position
    # Open the log file and seek to the last position
    with open(latest_log, "r") as file:
        file.seek(file_position)
        # Read new lines
        new_lines = file.readlines()
        file_position = file.tell()
        # Process lines containing "iteration"
        last_match = None
        for line in new_lines:
            match = iteration_pattern.search(line)
            if match:
                last_match = match
        if last_match:
            # Extract the iteration count from the regex match
            iteration_count = int(last_match.group(1))
            print(f"{iteration_count} iterations")

# Read the PID from the file
with open("train_process.pid", "r") as f:
    pid = int(f.read().strip())
    print("PID:", pid)

# Function to check if the process is running
def is_process_running(pid):
    try:
        os.kill(pid, 0)  # Sending signal 0 to check if the process exists
        return True
    except OSError:
        return False

# Monitor the training process
while is_process_running(pid):
    read_new_iterations()
    print("Training is still running...")
    time.sleep(30)  # Check every X seconds

print("Training has finished.")


PID: 7889
Training is still running...
40 iterations
Training is still running...
80 iterations
Training is still running...
120 iterations
Training is still running...
160 iterations
Training is still running...
200 iterations
Training is still running...


In [9]:
# Here we could disconnect from the GPU resource
#from google.colab import runtime
#runtime.unassign()